In [1]:
import os
import sys
import random

import copy 
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2

In [2]:
import segmentation_models as sm

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [3]:
from tensorflow.keras import backend as K

In [4]:
import pydicom as dicoms

In [5]:
sm.set_framework('tf.keras')

In [6]:
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

In [9]:
train_path = r"J:\yasin\finalData"
checkPoint=r"J:\yasin\finalData\backup"
epochs = 30
batch_size = 8
image_size=512

data = pd.read_csv(r"J:\yasin\finalData\train.csv")

In [10]:
data.head()

,Unnamed: 0,patient,dicom_path,mask_path,size
0,0,abdullah_celik,finalData\img\abdullah_celik\56.jpg,finalData\mask\abdullah_celik\56.jpg,17641
1,1,Ahmet_Ozturk,finalData\img\Ahmet_Ozturk\28.jpg,finalData\mask\Ahmet_Ozturk\28.jpg,24082
2,2,Ahmet_Ozturk,finalData\img\Ahmet_Ozturk\29.jpg,finalData\mask\Ahmet_Ozturk\29.jpg,25368
3,3,Ahmet_Ozturk,finalData\img\Ahmet_Ozturk\30.jpg,finalData\mask\Ahmet_Ozturk\30.jpg,25961
4,4,ahmet_tamturk,finalData\img\ahmet_tamturk\18.jpg,finalData\mask\ahmet_tamturk\18.jpg,13545


In [11]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [10]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path)]    
    for goruntu in slices:
        goruntu.file_meta.TransferSyntaxUID = dicom.uid.ImplicitVRLittleEndian
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
#     scans.PixelData=img.to_bytes()
    image = scans.pixel_array
#     image = image.reshape([image.shape[1], image.shape[2], 3])
    #scans.PixelData#np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans.RescaleIntercept
    slope = scans.RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [11]:
from skimage import morphology
from skimage import measure
from skimage.filters import threshold_otsu, median
from scipy.ndimage import binary_fill_holes
from skimage.segmentation import clear_border
from scipy.stats import describe

In [12]:
def lung_segment(img, display=False):
    thresh = threshold_otsu(img)
    binary = img <= thresh

    lungs = median(clear_border(binary))
    lungs = morphology.binary_closing(lungs, selem=morphology.disk(7))
    lungs = binary_fill_holes(lungs)

    final = lungs*img
    final[final == 0] = np.min(img)
    
    return final


In [13]:
class DataGen(keras.utils.Sequence):
    def __init__(self, data, shuffle = False, batch_size=16,
                 image_size = 512,
                 preprocess=None, info={}):
        self.data = data               
        self.batch_size = batch_size 
        self.image_size = image_size        
        self.preprocess = preprocess        
        self.shuffle = shuffle              
        self.on_epoch_end()
        
    def __getitem__(self, index):
        images = []
        masks  = []        
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        raw= self.data.iloc[indexes]
        for i,row in raw.iterrows(): 
            try:
                d_filepath=row["dicom_path"]
                goruntu=dicoms.dcmread(d_filepath, force = True)
                goruntu.file_meta.TransferSyntaxUID = dicoms.uid.ImplicitVRLittleEndian               
                imgs = get_pixels_hu(goruntu)
                seg = lung_segment(imgs)                
                seg = cv2.resize(seg, (self.image_size, self.image_size))
#                 print(seg.shape)
                images.append(seg)
                mask=cv2.imread(row["mask_path"], 1)
                mask = cv2.resize(mask, (self.image_size, self.image_size))
                masks.append(mask)
            except Exception as e:
                pass
#                 print(e)
        
        images = np.array(images)
        masks  = np.array(masks)
        
        if self.preprocess!=None: images = self.preprocess(images)
        
        return images,masks
        
    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)



In [14]:
image_size=128
data = data.sample(frac=1).reset_index(drop=True)
idx = int(0.8*len(data))
train_gen = DataGen(data.iloc[:idx], shuffle =True, batch_size=batch_size,   
                    image_size = image_size)
valid_gen = DataGen(data.iloc[idx:], batch_size=batch_size,   
                    image_size = image_size)

In [15]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c
def UNet():
    f = [128, 64, 32,16,8]
    inputs = keras.layers.Input((image_size, image_size, 1))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.MeanIoU(num_classes=1)])
# model.summary()


checkpoint = tf.keras.callbacks.ModelCheckpoint(checkPoint+"/model-{epoch:02d}-{acc:.2f}.hdf5", 
                                               monitor='acc', 
                                               verbose=1, 
                                               save_best_only=True, 
                                               mode='max')

callbacks = [checkpoint,
  tf.keras.callbacks.TensorBoard(
          log_dir='./log/{}'.format(
                  dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), 
                  write_images=True)]
    

    
history = model.fit_generator(train_gen,
                              validation_data=valid_gen, 
                    epochs=epochs
                  )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  assertion failed: [`labels` out of bound] [Condition x < y did not hold element-wise:] [x (confusion_matrix/control_dependency:0) = ] [0 0 0...] [y (confusion_matrix/Cast_2:0) = ] [1]
	 [[{{node confusion_matrix/assert_less/Assert/AssertGuard/else/_19/Assert}}]]
	 [[confusion_matrix/assert_less_1/Assert/AssertGuard/pivot_f/_31/_73]]
  (1) Invalid argument:  assertion failed: [`labels` out of bound] [Condition x < y did not hold element-wise:] [x (confusion_matrix/control_dependency:0) = ] [0 0 0...] [y (confusion_matrix/Cast_2:0) = ] [1]
	 [[{{node confusion_matrix/assert_less/Assert/AssertGuard/else/_19/Assert}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_2731]

Function call stack:
train_function -> train_function


In [ ]:
fig=plt.figure(figsize=(20, 6))
fig, axs = plt.subplots(1,2, figsize=(20, 6))
axs[1].plot(history.history['acc'], label='acc')
axs[1].plot(history.history['val_acc'],label='val_acc')

axs[1].legend()
axs[0].plot(history.history['loss'], label='loss')
axs[0].plot(history.history['val_loss'],label='val_loss')

axs[0].legend()

In [ ]:
BACKBONE = 'resnet18'
preprocess_input = sm.get_preprocessing(BACKBONE)
model = sm.Unet(BACKBONE, #encoder_weights='imagenet',
                classes=4,
                input_shape=(image_size, image_size, 3), 
                activation='softmax')

In [ ]:
LR = 0.0001
total_loss = sm.losses.dice_loss + (1 * sm.losses.categorical_focal_loss)
optim = keras.optimizers.Adam(LR)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5),dice_coef]
model.compile(
    optim,
    loss=total_loss,#sm.losses.dice_loss,#"binary_crossentropy"
    metrics=metrics,
)

In [ ]:
x, y = train_gen.__getitem__(0)
print(x[2].shape)
fig=plt.figure(figsize=(20, 6))
fig, axs = plt.subplots(1,2, figsize=(20, 6))
axs[0].imshow(x[5])
axs[1].imshow(y[5],cmap="gray")

In [ ]:
# x, y = train_gen.__getitem__(0)
p=model.predict(x)

In [ ]:
index=0
r=np.squeeze(p[index])
fig=plt.figure(figsize=(20, 6))
fig, axs = plt.subplots(1,3, figsize=(20, 6))
axs[0].imshow(x[index])
axs[1].imshow(y[index])
axs[2].imshow(r)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(checkPoint+'/best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]
    

In [ ]:
history = model.fit_generator(train_gen, 
                              validation_data=valid_gen,
                              steps_per_epoch=len(train_gen), 
                              validation_steps=len(valid_gen),
                              epochs=epochs,                              
                              callbacks=callbacks)
model.save_weights(r"backup\FPNresnet50-256_Models.h5")

In [ ]:
x, y = valid_gen.__getitem__(10)

In [ ]:
result = model.predict(x)
result = result > 0.5

In [ ]:
plt.imshow(np.reshape(result[0,:,:,2], (image_size, image_size)), cmap="gray")

In [ ]:
plt.imshow(y[0,:,:,2], cmap="gray")

In [ ]:
def mask2rle(img, x):   
    img = np.reshape(img, (image_size, image_size))    
    mask = Image.fromarray(img).resize(size=(x[0],x[1]))
    img=np.array(mask)
    pixels = img.T.flatten()#Look at the image after transposing    
    pixels = np.concatenate([[0], pixels, [0]])    
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1    
    runs[1::2] -= runs[::2]
    return runs
    #print("runs=",runs)


In [ ]:
sub=pd.read_csv("sample_submission.csv")
sub["file_name"] = sub["filename_class"].str.split("_").str[0]+".jpeg"
sub["encoded_mask"]=sub["encoded_mask"].astype(str)
sub.head()

In [ ]:
def mask2rle(img):
    pixels = img.T.flatten()
    pixels = np.pad(pixels, ((1, 1), ))
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
path=r"J:\anadolu\Testing_Imgs\Testing_Imgs\\"    
for i,row in sub.iterrows():
    try:
        x = Image.open(path+sub.loc[i,"file_name"])
        img = np.array(x.resize((image_size,image_size)))
        img = np.expand_dims(img, axis=0)
        result = model.predict(img)
        result = result > 0.5
        index = int(row["filename_class"][-1])      
        result = np.reshape(result[0,:,:,index-1], (image_size,image_size))
        result = Image.fromarray(result).resize(size=(x.size[1],x.size[0]))
        result = np.array(result)
        a=mask2rle(result.round().astype(int))
        sub.at[i,"encoded_mask"]=a
        x.close()        
    except:
        pass


In [ ]:
sub=sub.drop(["file_name"], axis=1)
sub.to_csv("backup\sample_submission.csv", index=False)

In [ ]:
sub["encoded_mask"]=sub["encoded_mask"].astype(str)
sub.info()

In [ ]:
file=r"J:\anadolu\Testing_Imgs\Testing_Imgs\1f374b72-0953-42da-9919-843c765c1499.jpeg"
x = Image.open(file)
plt.imshow(x)
img=np.array(x.resize((image_size,image_size)))
img = np.expand_dims(img, axis=0)
print(img.shape)


In [ ]:
img = np.reshape(result[0,:,:,1], (256, 256))
mask = Image.fromarray(img).resize(size=(256,256))
img=np.array(mask)
plt.imshow(img)


In [ ]:
a=mask2rle(img.round().astype(int))

In [ ]:
sub=pd.read_csv("backup\sample_submission.csv")

In [ ]:
sub.info()